In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('practice-347601-6e0b822fd842.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#date = 0 #当日は0、前日は1・・・
#
date = 1 #当日は0、前日は1・・・
date2 = "08/08"
pmc = 23089001 # 店舗ID
SPREADSHEET_KEY = '1uDKyi_DPZvNo9xfmCOgRkpLrEBoS-BPvcw34rb2e7WE' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 801 # スロットの1台目
wait = 0.01

user_id = "g84909252+003@gmail.com"
user_pw = "sitega22"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }
headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/08', '025489', '1', '42051.0', '-21967.0', '4', '3']
['08/08', '025489', '2', '35897.0', '-10531.0', '9', '5']
['08/08', '025489', '5', '45641.0', '7825.0', '22', '7']
['08/08', '025489', '6', '45641.0', '-21666.0', '6', '4']
['08/08', '025489', '7', '34872.0', '-4513.0', '12', '4']
['08/08', '025489', '8', '33333.0', '9631.0', '19', '5']
['08/08', '025579', '10', '25897.0', '2107.0', '13', '4']
['08/08', '025737', '11', '33077.0', '-15346.0', '23', '5']
['08/08', '025737', '12', '20256.0', '-1203.0', '28', '3']
['08/08', '025737', '13', '20000.0', '302.0', '36', '2']
['08/08', '025364', '15', '33333.0', '-4212.0', '33', '12']
['08/08', '025404', '16', '28205.0', '904.0', '32', '8']
['08/08', '025404', '17', '32308.0', '-4814.0', '26', '13']
['08/08', '025603', '18', '23846.0', '3612.0', '26', '9']
['08/08', '025603', '20', '55641.0', '-4814.0', '44', '22']
['08/08', '025665', '21', '53077.0', '-16851.0', '20', '4']
['08/08', '025665', '22', '43846.0', '38820.0', '53', '12']
['08/

['08/08', '025873', '215', '15897.0', '-11735.0', '10', '4']
['08/08', '025818', '216', '23846.0', '-6619.0', '9', '2']
['08/08', '025818', '217', '30513.0', '-25277.0', '3', '3']
['08/08', '025818', '218', '29231.0', '-20763.0', '4', '4']
['08/08', '025818', '220', '24359.0', '1205.0', '16', '4']
['08/08', '025876', '221', '21282.0', '-15647.0', '4', '4']
['08/08', '025876', '222', '16410.0', '-6018.0', '9', '2']
['08/08', '025876', '223', '15641.0', '-8124.0', '4', '1']
['08/08', '025824', '225', '13333.0', '2709.0', '18', '2']
['08/08', '025824', '226', '17436.0', '-11133.0', '5', '1']
['08/08', '025882', '227', '3333.0', '-2707.0', '0', '0']
['08/08', '025882', '228', '3846.0', '-3008.0', '1', '1']
['08/08', '025882', '230', '8205.0', '19561.0', '76', '3']
['08/08', '025879', '231', '21282.0', '-7522.0', '12', '4']
['08/08', '025850', '232', '9487.0', '-7522.0', '3', '3']
['08/08', '025797', '233', '5385.0', '12339.0', '55', '4']
['08/08', '025847', '235', '9744.0', '-6018.0', '13'

['08/08', '025665', '528', '40256.0', '-11133.0', '19', '10']
['08/08', '025665', '530', '45128.0', '9631.0', '33', '8']
['08/08', '025665', '531', '45128.0', '3612.0', '29', '8']
['08/08', '025665', '532', '44359.0', '-16550.0', '17', '7']
['08/08', '025665', '533', '52308.0', '7223.0', '34', '7']
['08/08', '025665', '535', '40513.0', '-11735.0', '16', '4']
['08/08', '025665', '536', '47692.0', '-12939.0', '20', '5']
['08/08', '025665', '537', '48974.0', '34908.0', '49', '6']
['08/08', '025665', '538', '44359.0', '42131.0', '53', '8']
['08/08', '025665', '550', '47436.0', '-4513.0', '25', '8']
['08/08', '025665', '551', '50256.0', '9631.0', '35', '8']
['08/08', '025665', '552', '54615.0', '2709.0', '33', '4']
['08/08', '025665', '553', '47436.0', '-12036.0', '20', '5']
['08/08', '025665', '555', '50513.0', '3010.0', '32', '7']
['08/08', '025665', '556', '47179.0', '8427.0', '33', '8']
['08/08', '025665', '557', '44103.0', '17756.0', '36', '5']
['08/08', '025665', '558', '33333.0', '-2

['08/08', '025825', '751', '13333.0', '3913.0', '10', '1']
['08/08', '025825', '752', '25897.0', '-11434.0', '7', '3']
['08/08', '025839', '753', '14615.0', '9631.0', '19', '3']
['08/08', '025837', '755', '6923.0', '-1504.0', '4', '1']
['08/08', '025849', '756', '12564.0', '13242.0', '16', '3']
['08/08', '025849', '757', '5385.0', '-3309.0', '1', '1']
['08/08', '025849', '758', '9744.0', '3913.0', '8', '2']
['08/08', '025849', '760', '15641.0', '6320.0', '13', '3']
['08/08', '025836', '761', '9744.0', '-1805.0', '5', '2']
['08/08', '025836', '762', '4615.0', '-1504.0', '2', '2']
['08/08', '025836', '763', '4872.0', '-4513.0', '0', '0']
['08/08', '025836', '765', '24615.0', '302.0', '15', '2']
['08/08', '025842', '766', '2821.0', '-2406.0', '0', '0']
['08/08', '025842', '767', '6923.0', '-4513.0', '1', '1']
['08/08', '025894', '768', '6667.0', '11135.0', '11', '2']
['08/08', '025894', '770', '15128.0', '904.0', '9', '4']
['08/08', '025489', '771', '45128.0', '40325.0', '41', '8']
['08/0

['08/08', '120122', '813', '7453', '-4352.0', '50', '15', '--']
['08/08', '120122', '815', '6000', '-3704.0', '36', '12', '--']
['08/08', '120122', '816', '7284', '2685.0', '96', '34', '--']
['08/08', '120122', '817', '7107', '1528.0', '86', '27', '--']
['08/08', '120122', '818', '5887', '880.0', '72', '15', '--']
['08/08', '120122', '820', '6281', '185.0', '66', '21', '--']
['08/08', '120122', '871', '6644', '3380.0', '94', '23', '--']
['08/08', '120122', '872', '7883', '-2407.0', '71', '17', '--']
['08/08', '120122', '873', '5780', '-2824.0', '40', '12', '--']
['08/08', '120122', '875', '6779', '-1759.0', '64', '16', '--']
['08/08', '120122', '876', '5324', '5648.0', '98', '10', '--']
['08/08', '120122', '877', '6862', '-185.0', '73', '18', '--']
['08/08', '120122', '878', '8047', '-3056.0', '60', '19', '--']
['08/08', '120122', '880', '7155', '-4352.0', '42', '12', '--']
['08/08', '120122', '881', '7032', '2454.0', '88', '26', '--']
['08/08', '120122', '882', '6926', '1343.0', '92',

['08/08', '119996', '1266', '7609', '185.0', '29', '20', '--']
['08/08', '119996', '1267', '4659', '-1019.0', '14', '11', '--']
['08/08', '119996', '1268', '5066', '-880.0', '18', '10', '--']
['08/08', '119996', '1270', '6835', '1250.0', '32', '19', '--']
['08/08', '119996', '1271', '5069', '-1204.0', '14', '14', '--']
['08/08', '119996', '1272', '7411', '972.0', '32', '24', '--']
['08/08', '120110', '1273', '6125', '-1620.0', '30', '16', '--']
['08/08', '120110', '1275', '5572', '-1157.0', '29', '17', '--']
['08/08', '120110', '1276', '5686', '-3009.0', '22', '10', '--']
['08/08', '120110', '1277', '5595', '-3472.0', '19', '14', '--']
['08/08', '120110', '1278', '5007', '2731.0', '42', '14', '--']
['08/08', '120110', '1280', '4671', '-3241.0', '15', '9', '--']
['08/08', '120110', '1281', '5633', '6620.0', '61', '27', '--']
['08/08', '120110', '1282', '6247', '-1759.0', '33', '13', '--']
['08/08', '120110', '1283', '6416', '-4676.0', '19', '12', '--']
['08/08', '120110', '1285', '5653'

['08/08', '120141', '1567', '3617', '-694.0', '12', '7', '--']
['08/08', '120141', '1568', '9286', '1389.0', '37', '40', '--']
['08/08', '120141', '1570', '8072', '1435.0', '41', '20', '--']
['08/08', '120141', '1571', '8936', '1204.0', '35', '37', '--']
['08/08', '120141', '1572', '7739', '926.0', '32', '24', '--']
['08/08', '120141', '1573', '4194', '-1528.0', '12', '11', '--']
['08/08', '120141', '1575', '5946', '-463.0', '20', '17', '--']
['08/08', '120141', '1576', '8087', '1250.0', '38', '25', '--']
['08/08', '120141', '1577', '9469', '1991.0', '41', '38', '--']
['08/08', '120141', '1578', '3878', '-1389.0', '10', '10', '--']
['08/08', '120137', '1580', '7290', '-509.0', '24', '6', '30']
['08/08', '120137', '1581', '8374', '-231.0', '23', '23', '46']
['08/08', '120137', '1582', '7646', '-2315.0', '16', '20', '36']
['08/08', '120137', '1583', '5239', '-1111.0', '14', '9', '23']
['08/08', '119984', '1585', '7044', '-139.0', '25', '15', '--']
['08/08', '120036', '1586', '6227', '-11

['08/08', '120153', '2006', '7918', '-417.0', '47', '0', '39']
['08/08', '120122', '2007', '6750', '1620.0', '81', '28', '--']
['08/08', '120145', '2008', '8414', '5093.0', '0', '14', '114']
['08/08', '120145', '2010', '8378', '-93.0', '0', '13', '83']
['08/08', '120145', '2011', '6946', '-185.0', '0', '11', '63']
['08/08', '120145', '2012', '4997', '-3519.0', '0', '8', '26']
['08/08', '120145', '2013', '7506', '-185.0', '0', '13', '71']
['08/08', '120145', '2015', '9468', '93.0', '0', '16', '93']
['08/08', '120145', '2016', '7189', '-3704.0', '0', '15', '47']
['08/08', '120148', '2017', '9203', '-1991.0', '0', '14', '58']
['08/08', '120148', '2018', '9554', '6296.0', '0', '21', '93']
['08/08', '120148', '2020', '9386', '7731.0', '0', '16', '113']
['08/08', '120122', '2021', '6442', '-2176.0', '52', '15', '--']
['08/08', '120122', '2022', '7306', '4769.0', '99', '28', '--']
['08/08', '120122', '2023', '6121', '-648.0', '60', '18', '--']
['08/08', '120122', '2025', '5679', '9676.0', '99

{'spreadsheetId': '1uDKyi_DPZvNo9xfmCOgRkpLrEBoS-BPvcw34rb2e7WE',
 'updates': {'spreadsheetId': '1uDKyi_DPZvNo9xfmCOgRkpLrEBoS-BPvcw34rb2e7WE',
  'updatedRange': 'S5RAW!F5657:M5688',
  'updatedRows': 32,
  'updatedColumns': 8,
  'updatedCells': 256}}